We expect T-cell and NK cell populations to be effected by the STAT4 mutation, so we will specifically look into those subpopulations. 

We also want to  look at disease-effects (Patient 2, off treatment vs control) and treatment-effects (Patient 1, on treatment vs Patient 2, off treatment). 

[CACOA](https://doi.org/10.1101/2022.03.15.484475) is specifically developed for same cell type DE across samples, so we will use it for these test. 

In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat, quietly = T)
        library(openxlsx, quietly = T)
        library(cacoa, quietly = T)
    })
})

data_path = '/data3/hratch/STAT4_v2/'
n.cores=20

In [2]:
pbmc.integrated<-readRDS(paste0(data_path, 'processed/pbmc_integrated.RDS'))
md<-pbmc.integrated@meta.data

Specifiy the cell types and context comparisons to test for:

In [3]:
cell.types<-c('Naive CD8+ T cells', 'Natural killer  cells', 
              'Naive CD4+ T cells', 'Effector CD4+ T cells', 'Memory CD4+ T cells')
comparisons<-list(disease.effect = list(target = 'Patient.2', ref = 'Control'), 
                treatment.effect = list(target = 'Patient.1', ref = 'Patient.2'))

# CACOA

In [ ]:
# wilcoxon.de<-function(cell.type, context.treat, context.base){
#     pbmc.subset<-subset(x = pbmc.integrated, subset = Cell.Type == ct)
#     Idents(pbmc.subset)<-'orig.ident'

#     de.res<-FindMarkers(object = pbmc.subset, 
#                         ident.1 = context.treat, ident.2 = context.base,
#                         assay = 'RNA', only.pos = F, 
#                         slot = 'data', test.use = 'wilcox', 
#                         min.pct = 0.1, # default
#                         logfc.threshold = 0.5 # default
#                                       )
#     de.res[['gene']]<-rownames(de.res)
#     de.res[['Cell.Type']]<-ct
#     de.res[['Comparison']]<-paste0(context.treat, '_vs_', context.base)
    
#     return(de.res)
# }

Since the CACOA pacakge uses the DESeq2 package, but with a latent covariate calculated, we will use the raw UMI counts

In [68]:
comparison<-comparisons[[1]]
cell.type<-'Effector CD4+ T cells'
pbmc.subset = subset(pbmc.integrated, 
                     subset = (orig.ident %in% comparison) & (Cell.Type == cell.type))
cao <- Cacoa$new(data.object=pbmc.subset@assays$RNA@counts, 
                 n.cores=n.cores,
                 sample.groups=droplevels(pbmc.subset$orig.ident), 
                 cell.groups=droplevels(pbmc.subset$Cell.Type), 
                 sample.per.cell=droplevels(pbmc.subset$orig.ident), 
                 ref.level=comparison$ref, 
                 target.level=comparison$target)

cao$estimateDEPerCellType(cell.groups=cell.type, 
#                           test =  'edgeR', # benchmarked
                         resampling.method='loo', 
                         min.cell.frac=0.1 # same as Seurat default
                         )

Warning message in initialize(...):
“Many function may be not supported for an object of class dgCMatrix”
Interpreting data.object as a raw count matrix

Warning message in mclapply(X, FUN, ..., mc.cores = mc.cores, mc.preschedule = mc.preschedule, :
“all scheduled cores encountered errors in user code”


In [69]:
head(droplevels(pbmc.subset$orig.ident))

AAAGGATTCTTTGATC-1 .Control AAAGTCCCAGAGACTG-1 .Control 
                    Control                     Control 
AACAAGAGTGCAATGG-1 .Control AACCCAATCCACACCT-1 .Control 
                    Control                     Control 
AACGAAAGTGAATTAG-1 .Control AACGTCAGTCAAGGCA-1 .Control 
                    Control                     Control 
Levels: Control Patient.2